## Importing libraries and PFR geometry

In [1]:
import numpy as np
import math
from PDESystems.pfr_pde import PfrIpoptEnv
import torch
from Encoders.pca_model import PcaWarmLibrary
from Encoders.autoencoder import AeWarmLibrary, PrimalsAutoencoder
from Encoders.variational_autoncoder import VaeWarmLibrary, PrimalsVAE
from Encoders.regressor_train import LatentRegressor
from SACAgent.sac_agent import SACAgent
from rl_helper.train_fn import run_td3_with_baseline

In [2]:
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
D_tube = 0.126
A_t = math.pi * (D_tube / 2.0) ** 2

params = {
    # geometry
    "A_t":   A_t,

    # inlet
    "cA_in": 80.0,
    "cB_in": 0.0,
    "cC_in": 0.0,
    "T_in": 423.0,

    # kinetics
    "Rgas": 8.314,
    "k01": 0.1, # 1/h
    "E1": 33475.0, # J/mol
    "k02": 9.0e-7, # m^3/(mol*h)
    "E2": 75319.0, # J/mol

    # heats of reaction (exothermic)
    "dH1": -20.0e3, # J/mol
    "dH2": -60.0e3, # J/mol

    # heat transfer per volume + volumetric heat capacity
    "Ua": 8524.0 * 1e3, # J/(m^3*h*K)
    "rhoCp": 8.0e5, # J/(m^3*K)

    # MV bounds
    "F_min": 0.6,
    "F_max": 4.5,
    "Tj_min": 298.0,
    "Tj_max": 600.0,

    # path/safety bounds on Tr
    "T_lo": 280.0,
    "T_hi": 700.0,
}

In [4]:
# Coarse mesh
pca_env = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
)

In [5]:
# setpoints generation
num_episodes = 3000
np.random.seed(42)
setpoints = np.random.uniform(20.0, 55.0, size=num_episodes)

# TD3 and Reward Config

In [6]:
STATE_DIM = 19
ACTION_DIM = 5  # [a_warm, a_pca, a_mu, a_tol, a_atol]
ACTOR_LAYER_SIZES = [256, 256, 256, 256]
CRITIC_LAYER_SIZES = [256, 256, 256, 256]
BUFFER_CAPACITY = 200_000
ACTOR_LR = 1e-4
CRITIC_LR = 1e-4
ALPHA_LR = 1e-4
SMOOTHING_STD = 0.2
NOISE_CLIP = 0.5
GAMMA = 0.99
TAU = 0.01
MAX_ACTION = 1.0
POLICY_DELAY = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128
STD_START = 0.2
STD_END = 0.01
STD_DECAY_RATE = 0.99995
STD_DECAY_MODE = "exp"

In [7]:
REWARD_CFG = {
    # feasibility / reliability
    "R_fail_vs_base": 500.0, # baseline solves, RL fails
    "R_rescue": 500.0, # baseline fails, RL solves
    "R_both_fail": 50.0, # both fail

    # efficiency weights
    "w_iter": 50.0, # weight on iteration gain
    "w_time": 10.0, # weight on time gain

    # solution quality (tracking/objective) weight
    "w_L": 1.0,

    # safety (temperature margin) weight
    "w_Tsafe": 1.0,

    # KKT-ish weight (multipliers/active bounds)
    "w_kkt": 0.5,

    # action regularization
    "w_act": 1e-3,   # penalty on |a|^2
    "w_act_delta": 1e-3, # penalty on |a_k - a_{k-1}|^2
}

In [8]:
def make_sac_agent():
    return SACAgent(
        state_dim=STATE_DIM,
        action_dim=ACTION_DIM,
        actor_hidden=ACTOR_LAYER_SIZES,
        critic_hidden=CRITIC_LAYER_SIZES,
        gamma=GAMMA,
        actor_lr=ACTOR_LR,
        critic_lr=CRITIC_LR,
        alpha_lr=ALPHA_LR,
        batch_size=BATCH_SIZE,
        max_action=MAX_ACTION,
        tau=TAU,
        buffer_size=BUFFER_CAPACITY,
        device=DEVICE,
        use_per=False,
    )

## Without optimal solutions

In [9]:
sac_agent = make_sac_agent()

In [10]:
env_rl = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=None,
)

env_base = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=None,
)

stats = run_td3_with_baseline(
    env_rl=env_rl,
    env_base=env_base,
    agent=sac_agent,
    setpoints=setpoints,
    warm_start_steps=100,
    cfg=REWARD_CFG,
    tee=False,
)

Initial setpoint s0=33.11 solved; starting RL stream with baseline...
[step 100/2999] s=21.10, r=21.1, iters_rl=15, iters_base=23, time_rl=0.174s, time_base=0.277s | w_warm=0.94, w_pca=0.00, mu=3.49e-04, tol=3.1e-05
[step 200/2999] s=42.47, r=40.4, iters_rl=7, iters_base=25, time_rl=0.157s, time_base=0.278s | w_warm=0.97, w_pca=0.00, mu=1.21e-04, tol=3.3e-05
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.14.19\x3a Maximum Number of Iterations
      Exceeded.
[PFR ENV] IPOPT failed or returned non-optimal status at s=31.83
[step 300/2999] s=21.81, r=8.9, iters_rl=15, iters_base=18, time_rl=0.160s, time_base=0.169s | w_warm=0.97, w_pca=0.00, mu=1.07e-04, tol=9.4e-05
[step 400/2999] s=23.61, r=37.1, iters_rl=18, iters_base=47, time_rl=0.180s, time_base=0.477s | w_warm=0.99, w_pca=0.00, mu=1.20e-04, tol=7.9e-05
[step 500/2999] s=44.44, r=34.8, iters_rl=13, iters_base=32, time_rl=0.184s, time_base=0.375s | w_warm=0.95, w_pca=0.00, mu=1.03

In [11]:
# Saving
sac_agent.save("checkpoints", prefix="sac_no_optimal")

Saved checkpoint to: checkpoints\sac_no_optimal_20251202_023108.pkl


## Using PCA

In [12]:
# 1) Load library (PCA + scalers + Z_data)
pca_lib = PcaWarmLibrary.load(r"checkpoints\pca_warm_lib.joblib", coarse_env=pca_env)

# 2) Rebuild regressor with same dims
in_dim = pca_lib.feats.shape[1]
out_dim = pca_lib.Z_latent.shape[1]

pca_reg = LatentRegressor(in_dim=in_dim, out_dim=out_dim, hidden_dims=(64, 64))
state_dict = torch.load(r"checkpoints\pca_latent_regressor.pth", map_location=device)
pca_reg.load_state_dict(state_dict)
pca_reg.to(device).eval()

# 3) Attach regressor to library
pca_lib.attach_regressor(pca_reg, device=device)

In [13]:
sac_agent = make_sac_agent()

In [14]:
env_rl = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=pca_lib,
)

env_base = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=None,
)

stats = run_td3_with_baseline(
    env_rl=env_rl,
    env_base=env_base,
    agent=sac_agent,
    setpoints=setpoints,
    warm_start_steps=100,
    cfg=REWARD_CFG,
    tee=False,
)

Initial setpoint s0=33.11 solved; starting RL stream with baseline...
[step 100/2999] s=21.10, r=-15.3, iters_rl=28, iters_base=23, time_rl=0.384s, time_base=0.276s | w_warm=0.45, w_pca=0.24, mu=6.91e-02, tol=1.6e-08
[step 200/2999] s=42.47, r=34.5, iters_rl=10, iters_base=25, time_rl=0.150s, time_base=0.275s | w_warm=0.27, w_pca=0.21, mu=1.03e-04, tol=6.8e-05
[step 300/2999] s=21.81, r=27.8, iters_rl=9, iters_base=18, time_rl=0.141s, time_base=0.196s | w_warm=0.09, w_pca=0.66, mu=1.07e-04, tol=7.7e-05
[step 400/2999] s=23.61, r=39.3, iters_rl=16, iters_base=47, time_rl=0.175s, time_base=0.474s | w_warm=0.09, w_pca=0.66, mu=1.08e-04, tol=9.6e-05
[step 500/2999] s=44.44, r=41.9, iters_rl=9, iters_base=32, time_rl=0.153s, time_base=0.377s | w_warm=0.09, w_pca=0.66, mu=1.02e-04, tol=6.9e-05
[step 600/2999] s=25.91, r=23.4, iters_rl=9, iters_base=16, time_rl=0.139s, time_base=0.165s | w_warm=0.09, w_pca=0.66, mu=1.01e-04, tol=9.6e-05
[step 700/2999] s=38.64, r=39.5, iters_rl=9, iters_base=

In [15]:
# Saving
sac_agent.save("checkpoints", prefix="sac_pca")

Saved checkpoint to: checkpoints\sac_pca_20251202_025720.pkl


## Using AE

In [16]:
# 1) load library
ae_lib = AeWarmLibrary.load(r"checkpoints\ae_warm_lib.joblib", coarse_env=pca_env)

# 2) rebuild AE model with same sizes
input_dim = ae_lib.scaler.mean_.shape[0]     # D (number of primals)
latent_dim = ae_lib.Z_latent.shape[1]        # r (latent dimension)

ae_model = PrimalsAutoencoder(input_dim=input_dim, latent_dim=latent_dim)
ae_model.load_state_dict(torch.load(r"checkpoints\ae_model.pth", map_location=device))
ae_model.to(device).eval()

# 3) rebuild and attach regressor
in_dim = ae_lib.feats.shape[1]       # 3
out_dim = ae_lib.Z_latent.shape[1]   # r
ae_reg = LatentRegressor(in_dim=in_dim, out_dim=out_dim, hidden_dims=(64, 64))
ae_reg.load_state_dict(torch.load(r"checkpoints\ae_latent_regressor.pth", map_location=device))
ae_reg.to(device).eval()

ae_lib.attach_regressor(ae_reg, device=device)
ae_lib.attach_decoder(ae_model.decoder)

In [17]:
sac_agent = make_sac_agent()

In [18]:
env_rl = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=ae_lib,
)

env_base = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=None,
)

stats = run_td3_with_baseline(
    env_rl=env_rl,
    env_base=env_base,
    agent=sac_agent,
    setpoints=setpoints,
    warm_start_steps=100,
    cfg=REWARD_CFG,
    tee=False,
)

Initial setpoint s0=33.11 solved; starting RL stream with baseline...
[step 100/2999] s=21.10, r=21.1, iters_rl=15, iters_base=23, time_rl=0.173s, time_base=0.276s | w_warm=0.38, w_pca=0.41, mu=2.57e-03, tol=1.9e-08
[step 200/2999] s=42.47, r=32.6, iters_rl=11, iters_base=25, time_rl=0.150s, time_base=0.277s | w_warm=0.23, w_pca=0.21, mu=1.06e-04, tol=4.7e-05
[step 300/2999] s=21.81, r=30.5, iters_rl=8, iters_base=18, time_rl=0.127s, time_base=0.175s | w_warm=0.10, w_pca=0.65, mu=1.01e-04, tol=9.0e-05
[step 400/2999] s=23.61, r=40.5, iters_rl=15, iters_base=47, time_rl=0.170s, time_base=0.477s | w_warm=0.10, w_pca=0.66, mu=1.01e-04, tol=1.7e-08
[step 500/2999] s=44.44, r=42.0, iters_rl=9, iters_base=32, time_rl=0.149s, time_base=0.377s | w_warm=0.10, w_pca=0.63, mu=1.03e-04, tol=9.3e-05
[step 600/2999] s=25.91, r=19.2, iters_rl=10, iters_base=16, time_rl=0.149s, time_base=0.156s | w_warm=0.10, w_pca=0.63, mu=1.46e-04, tol=1.5e-07
[step 700/2999] s=38.64, r=36.5, iters_rl=11, iters_base

In [19]:
# Saving
sac_agent.save("checkpoints", prefix="sac_ae")

Saved checkpoint to: checkpoints\sac_ae_20251202_032400.pkl


## Using VAE

In [20]:
vae_lib = VaeWarmLibrary.load(r"checkpoints\vae_warm_lib.joblib", coarse_env=pca_env)

input_dim = vae_lib.scaler.mean_.shape[0]
latent_dim = vae_lib.Z_mu.shape[1]

vae_model = PrimalsVAE(in_dim=input_dim, latent_dim=latent_dim)
vae_model.load_state_dict(torch.load(r"checkpoints\vae_model.pth", map_location=device))
vae_model.to(device).eval()

in_dim = vae_lib.feats.shape[1]   # 3
out_dim = vae_lib.Z_mu.shape[1]   # latent_dim
vae_reg = LatentRegressor(in_dim=in_dim, out_dim=out_dim, hidden_dims=(64, 64))
vae_reg.load_state_dict(torch.load(r"checkpoints\vae_latent_regressor.pth", map_location=device))
vae_reg.to(device).eval()

vae_lib.attach_regressor(vae_reg, device=device)
vae_lib.attach_decoder(vae_model.decoder)

[VAE LIB] Loaded from checkpoints\vae_warm_lib.joblib, feats shape=(5000, 3), X_recon shape=(5000, 966)


In [21]:
sac_agent = make_sac_agent()

In [22]:
env_rl = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=vae_lib,
)

env_base = PfrIpoptEnv(
    params=params,
    N=10,
    K=20,
    L=20.0,
    dt=0.02,
    target_species="B",
    n_zones=5,
    max_iter=500,
    pca_lib=None,
)

stats = run_td3_with_baseline(
    env_rl=env_rl,
    env_base=env_base,
    agent=sac_agent,
    setpoints=setpoints,
    warm_start_steps=100,
    cfg=REWARD_CFG,
    tee=False,
)

Initial setpoint s0=33.11 solved; starting RL stream with baseline...
[step 100/2999] s=21.10, r=6.4, iters_rl=20, iters_base=23, time_rl=0.280s, time_base=0.277s | w_warm=0.23, w_pca=0.25, mu=5.77e-04, tol=5.6e-06
[step 200/2999] s=42.47, r=32.6, iters_rl=11, iters_base=25, time_rl=0.148s, time_base=0.275s | w_warm=0.21, w_pca=0.25, mu=1.20e-04, tol=6.4e-06
[step 300/2999] s=21.81, r=26.3, iters_rl=9, iters_base=18, time_rl=0.153s, time_base=0.175s | w_warm=0.41, w_pca=0.43, mu=1.13e-04, tol=8.0e-05
[step 400/2999] s=23.61, r=42.9, iters_rl=13, iters_base=47, time_rl=0.156s, time_base=0.475s | w_warm=0.09, w_pca=0.66, mu=1.03e-04, tol=7.9e-05
model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Ipopt 3.14.19\x3a Maximum Number of Iterations
      Exceeded.
[PFR ENV] IPOPT failed or returned non-optimal status at s=38.31
[step 500/2999] s=44.44, r=37.1, iters_rl=12, iters_base=32, time_rl=0.158s, time_base=0.377s | w_warm=0.10, w_pca=0.65, mu=1.02

In [23]:
# Saving
sac_agent.save("checkpoints", prefix="sac_vae")

Saved checkpoint to: checkpoints\sac_vae_20251202_035023.pkl
